## Modeling 
Building Models: Linear Regression and Random Forest Regressor

## 3.1) Importing Essential Libraries

In [6]:
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## 3.2) Load processed data

In [2]:
df = pd.read_csv('../data/walmart_features.csv')

# Split data into target and features
y = df['weekly_sales']
x = df.drop(columns=['weekly_sales', 'date'])

### 3.3) Train Test Split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

### 3.4) First model: Linear Regression

In [8]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)

mae_lr = mean_absolute_error(y_test, y_pred_lr)
print("Linear Regression MAE:", mae_lr)

Linear Regression MAE: 64606.29525460997


### 3.5) Second model: Random Forest

In [10]:
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
print("Random Forest MAE:", mae_rf)

Random Forest MAE: 47568.30420229006


### 3.6) Save models

In [11]:
import joblib

joblib.dump(rf, "../models/random_forest.pkl")
joblib.dump(lr, "../models/linear_regression.pkl")

['../models/linear_regression.pkl']